# Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

2023-03-21 09:53:46.245963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
data_dir = path

train_ds = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

validation_ds = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

In [ ]:
class_names = [item.split("-")[1] for item in train_ds.class_names]
print(class_names)

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.applications.resnet50 as resnet50
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the ResNet50 model pre-trained on ImageNet
base_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Add a new classification head on top of the pre-trained ResNet50 base model
model = tf.keras.Sequential([
    layers.Input((224, 224, 3)),
    layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(120, activation='softmax')
])

# Define EarlyStopping, ModelCheckpoint and ReduceLROnPlateau callbacks
es = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)

# Apply the preprocess function to the training dataset
train_ds_prepro = train_ds.map(preprocess)
validation_ds_prepro = validation_ds.map(preprocess)

# Compile the model with the Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the preprocessed training dataset and validate using the validation dataset
history = model.fit(train_ds_prepro, 
                    validation_data=validation_ds_prepro, 
                    epochs=10,
                    callbacks=[es, mc, rlr],
                    verbose=1)


In [ ]:
test_dir = "/content/dogs/cropped/cropped/test"


test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32)

loss, accuracy = model.evaluate(test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

import matplotlib.pyplot as plt
plot_history(history)

from PIL import Image
import requests 
from io import BytesIO
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img


def getImage(url):
  '''
  Get an image provided its url and resize it
  '''
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  plt.imshow(img)
  img = img.resize((224, 224))
  return img


# def predict_breed(url, model):
#   '''
#   Provided a url and a CNN, will predict the breed of a dog
#   '''
#   img = getImage(url)
#   img = img_to_array(img)
#   img = img.reshape((-1, 224, 224,3))
#   res = model.predict(img)[0][0]

url = "https://ds1.static.rtbf.be/article/image/370x208/7/8/f/a20f02dfccd07952da54a7f9a82b3e89-1524657315.jpg"
img = getImage(url)
img = img_to_array(img)

img = img.reshape((-1, 224, 224,3))
imp_prepro = train_ds.map(preprocess)
res = model.predict(imp_prepro)[0][0]
prob = 1 - res

import numpy as np
np.argmax(model.predict(img)[0])

class_names[97]